In [1]:
#Routine stuff
import numpy as np
import pandas as pd
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report, confusion_matrix

In [2]:
#Importing the training dataset into dataframe
df = pd.read_csv("train.csv",delimiter=",")
df = df.drop('Unnamed: 0',axis=1)
df.CLASSIFICATION = pd.Categorical(df.CLASSIFICATION)
df['CODE'] = df.CLASSIFICATION.cat.codes #Converting classes (names) to codes
df = df.drop('CLASSIFICATION',axis=1)
df.head()

,apr,gmt,1993,writes,references,article,sender,people,university,1,...,finland,bikes,livesey@solntze.wpd.sgi.com,shit,spot,justify,explained,52,learning,CODE
0,2,2,2,1,1,1,1,2,1,0,...,0,0,0,0,0,0,0,0,0,17
1,1,1,1,1,1,0,1,1,0,0,...,0,0,0,0,0,0,0,0,0,17
2,1,1,1,2,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,17
3,1,1,1,2,1,2,0,0,0,0,...,0,0,0,0,0,0,0,0,0,17
4,0,1,1,1,1,0,1,2,0,0,...,0,0,0,0,0,0,0,0,0,17


In [3]:
data = df.values #training dataframe to numpy array
data

array([[ 2,  2,  2, ...,  0,  0, 17],
       [ 1,  1,  1, ...,  0,  0, 17],
       [ 1,  1,  1, ...,  0,  0, 17],
       ..., 
       [ 1,  1,  0, ...,  0,  1, 19],
       [ 1,  1,  1, ...,  0,  0, 19],
       [ 1,  1,  1, ...,  0,  0, 19]])

In [4]:
y_train = data[:,-1] #separating ouput
y_train

array([17, 17, 17, ..., 19, 19, 19])

In [5]:
x_train = data[:,:-1] #separating input
x_train.shape

(15997, 2000)

In [6]:
#FUNCTION TO FIT THE TRAINING DATA INTO THE MODEL & RETURN RESULTANT DICTIONARY
def fit (x_train, y_train):
    result = {}
    class_values = set(y_train) #set of distinct class values
    for current_class in class_values: #iterate over each class among all classes
        result[current_class] = {}
        current_class_rows = (y_train == current_class) #all row numbers of the current class considered
        x_train_current = x_train[current_class_rows] #all rows of the current class considered (using row numbers)
        total_count = 0 #count of total words appeared
        num_words = x_train.shape[1]
        for j in range(1, num_words+1): #iterate over all words in the vocab
            result[current_class][j] = x_train_current[:,j-1].sum() #count of number of times word j appears in current class of docs
            total_count += result[current_class][j]
        result[current_class]["total_count"] = total_count
    return result

In [7]:
#FUNCTION TO RETURN PROBABLITY OF THE DOCUMENT BELONGING TO THE CURRENT_CLASS OF DOCUMENTS SELECTED
def probability(dictionary, x, current_class):
    output = 1
    num_words = len(dictionary[current_class].keys())-1 #subtracting 1 for "total_count" attribute
    for j in range(1, num_words+1): #iterate over all words in the vocab
        if(x[j-1]!=0): #multiply (add in log) the probability of the current word when the word is present in the document
            count_current_class_current_word = dictionary[current_class][j] + 1
            count_current_class = dictionary[current_class]["total_count"] + num_words
            current_probability = np.log(count_current_class_current_word) - np.log(count_current_class)
            output = output + current_probability
    #print(output)
    return output

In [8]:
#FUNCTION TO PREDICT THE CLASS OF THE CURRENT DOCUMENT SELECTED
def predictSinglePoint(dictionary, x):
    classes = dictionary.keys()
    first_run = True
    for current_class in classes:
        if (current_class == "total_data"):
            continue
        p_current_class = probability(dictionary, x, current_class) #probability of belonging to the current_class of documents
        if (first_run or p_current_class > best_p):
            #assign/update best_class if running for the 1st time or if best probability yet
            best_p = p_current_class
            best_class = current_class
            first_run = False
    return best_class

In [9]:
#FUNCTION THAT PREDICTS THE CLASS OF THE TEST DATASET
def predict(dictionary, x_test):
    y_pred = []
    for x in x_test:
        x_class = predictSinglePoint(dictionary, x)
        y_pred.append(x_class)
    return y_pred

In [10]:
#importing test dataset just like done for train dataset
df_test = pd.read_csv("test.csv",delimiter=",")
df_test = df_test.drop('Unnamed: 0',axis=1)
df_test.CLASSIFICATION = pd.Categorical(df_test.CLASSIFICATION)
df_test['CODE'] = df_test.CLASSIFICATION.cat.codes
df_test = df_test.drop('CLASSIFICATION',axis=1)
data_test = df_test.values
y_test = data_test[:,-1]
x_test = data_test[:,:-1]

In [11]:
x_test.shape

(4001, 2000)

In [12]:
x_train.shape

(15997, 2000)

In [13]:
dictionary = fit(x_train,y_train)
dictionary

{0: {1: 866,
  2: 727,
  3: 626,
  4: 977,
  5: 765,
  6: 790,
  7: 372,
  8: 707,
  9: 344,
  10: 109,
  11: 376,
  12: 258,
  13: 103,
  14: 222,
  15: 222,
  16: 124,
  17: 125,
  18: 65,
  19: 822,
  20: 24,
  21: 0,
  22: 150,
  23: 79,
  24: 90,
  25: 42,
  26: 175,
  27: 121,
  28: 33,
  29: 153,
  30: 219,
  31: 60,
  32: 226,
  33: 75,
  34: 40,
  35: 0,
  36: 69,
  37: 7,
  38: 23,
  39: 21,
  40: 19,
  41: 103,
  42: 10,
  43: 51,
  44: 87,
  45: 323,
  46: 68,
  47: 110,
  48: 9,
  49: 52,
  50: 119,
  51: 89,
  52: 255,
  53: 109,
  54: 37,
  55: 95,
  56: 38,
  57: 47,
  58: 23,
  59: 16,
  60: 74,
  61: 21,
  62: 75,
  63: 168,
  64: 43,
  65: 63,
  66: 90,
  67: 184,
  68: 112,
  69: 48,
  70: 191,
  71: 19,
  72: 61,
  73: 37,
  74: 2,
  75: 41,
  76: 187,
  77: 34,
  78: 191,
  79: 33,
  80: 60,
  81: 53,
  82: 134,
  83: 9,
  84: 58,
  85: 12,
  86: 7,
  87: 63,
  88: 53,
  89: 23,
  90: 117,
  91: 43,
  92: 17,
  93: 51,
  94: 2,
  95: 49,
  96: 140,
  97: 308,
  98

In [16]:
y_train_pred = predict(dictionary, x_train)
print(classification_report(y_train,y_train_pred))
print(confusion_matrix(y_train,y_train_pred))

             precision    recall  f1-score   support

          0       0.78      0.89      0.83       800
          1       0.91      0.82      0.86       800
          2       0.93      0.88      0.90       800
          3       0.90      0.91      0.91       800
          4       0.91      0.95      0.93       800
          5       0.92      0.93      0.92       800
          6       0.86      0.92      0.89       800
          7       0.90      0.96      0.93       800
          8       0.92      0.97      0.94       800
          9       0.95      0.98      0.97       800
         10       0.99      0.94      0.97       800
         11       0.96      0.92      0.94       800
         12       0.86      0.93      0.89       800
         13       0.95      0.92      0.93       800
         14       0.93      0.93      0.93       800
         15       0.99      1.00      0.99       797
         16       0.77      0.93      0.84       800
         17       0.95      0.87      0.91   

In [14]:
y_pred = predict(dictionary, x_test)
print(classification_report(y_test,y_pred))
print(confusion_matrix(y_test,y_pred))

             precision    recall  f1-score   support

          0       0.76      0.88      0.82       200
          1       0.86      0.79      0.82       200
          2       0.91      0.81      0.85       200
          3       0.87      0.86      0.87       200
          4       0.88      0.94      0.91       200
          5       0.92      0.91      0.91       200
          6       0.84      0.88      0.86       200
          7       0.83      0.93      0.88       200
          8       0.91      0.96      0.94       200
          9       0.97      0.99      0.98       200
         10       1.00      0.95      0.97       200
         11       0.97      0.89      0.93       200
         12       0.78      0.90      0.83       200
         13       0.93      0.89      0.91       200
         14       0.90      0.94      0.92       200
         15       0.98      1.00      0.99       200
         16       0.73      0.90      0.81       200
         17       0.96      0.88      0.92   